## **Codsoft Data science Task 1**
Name : Danthamala Balaji Preetham



TITANIC SURVIVAL PREDICTION

Use the Titanic dataset to build a model that predicts whether a
passenger on the Titanic survived or not. This is a classic beginner
project with readily available data.
The dataset typically used for this project contains information
about individual passengers, such as their age, gender, ticket
class, fare, cabin, and whether or not they survived.

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
# checking for null values
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [20]:
df.Age.fillna(df['Age'].mean(),inplace=True)

In [21]:
Features= ["Pclass","Sex","Age","SibSp","Parch"]
X= df[Features]
y= df.Survived

In [22]:
# conerting text columns to numbers

X_dummies = pd.get_dummies(X['Sex'])
X_dummies = X_dummies.astype(int)
X_dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [23]:
new_X=pd.concat([X,X_dummies], axis='columns')

In [24]:
last_X = new_X.drop(['Sex','male'], axis='columns')
last_X

,Pclass,Age,SibSp,Parch,female
0,3,22.000000,1,0,0
1,1,38.000000,1,0,1
2,3,26.000000,0,0,1
3,1,35.000000,1,0,1
4,3,35.000000,0,0,0
...,...,...,...,...,...
886,2,27.000000,0,0,0
887,1,19.000000,0,0,1
888,3,29.699118,1,2,1
889,1,26.000000,0,0,0


In [25]:
df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [26]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [27]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV

In [16]:
model_params = {
    'svm' : {
        'model' : SVC(gamma='auto'),
        'params' : {
            'C' : [1,10,20],
            'kernel' : ['rbf','linear']
    }
     },
    'Random_forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators' : [1,5,10, 20, 40]
        }
    },
    'Logistic_regression' : {
        'model' : LogisticRegression(solver='liblinear',multi_class='auto'),
        'params' : {
            'C' : [1,10,20]
        }
    },
    'Knearest neighbor' : {
        'model' : KNeighborsClassifier(),
        'params' : {
            'n_neighbors' :[5,7,9,11,15]
        }
    },
    'Decision_tree' : {
        'model' :DecisionTreeClassifier(),
        'params' : {
            'criterion' : ['gini', 'entropy', 'log_loss']
        }
    },

}

In [15]:
scores =[]

for model_name , mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'], cv= 5, return_train_score = False)
    clf.fit(last_X, y)
    scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_
    })

# creating a dataframe to store score of all these different model to find the best model for our problem.
score_df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
score_df

,model,best_score,best_params
0,svm,0.802517,"{'C': 10, 'kernel': 'rbf'}"
1,Random_forest,0.814864,{'n_estimators': 10}
2,Logistic_regression,0.788990,{'C': 1}
3,Knearest neighbor,0.789015,{'n_neighbors': 5}
4,Decision_tree,0.805857,{'criterion': 'gini'}
